In [1]:
from rule_based_segmentation import mask_non_skin
from ita_extraction import ITA, empirical_classification, kinyananjui_classification
import os
import pandas as pd
import imageio as im
import time

In [2]:
def func(file_name,real_class):
    image = im.imread(file_name)
    direct_ita = ITA(image)
    skin,__ = mask_non_skin(image)
    segmented_ita = ITA(skin)
    
    #kinyananjui_class_direct_ita = kinyananjui_classification(direct_ita)
    #empirical_class_direct_ita = empirical_classification(direct_ita)
    kinyananjui_class_segmented_ita = kinyananjui_classification(segmented_ita)
    empirical_class_segmented_ita = empirical_classification(segmented_ita)
    
    #direct_kinyananjui_hit = 1 if kinyananjui_class_direct_ita == real_class else 0
    #direct_empirical_hit = 1 if empirical_class_direct_ita == real_class else 0
    kinyananjui_hit = 1 if kinyananjui_class_segmented_ita == real_class else 0
    empirical_hit = 1 if empirical_class_segmented_ita == real_class else 0
    
    
    #direct_kinyananjui_relaxed_hit = 1 if kinyananjui_class_direct_ita >= real_class-1 and kinyananjui_class_direct_ita <= real_class+1 else 0
    #direct_empirical_relaxed_hit = 1 if empirical_class_direct_ita >= real_class-1 and empirical_class_direct_ita <= real_class+1 else 0
    kinyananjui_relaxed_hit = 1 if kinyananjui_class_segmented_ita >= real_class-1 and kinyananjui_class_segmented_ita <= real_class+1 else 0
    empirical_relaxed_hit = 1 if empirical_class_segmented_ita >= real_class-1 and empirical_class_segmented_ita <= real_class+1 else 0
    
    return segmented_ita,direct_ita,kinyananjui_class_segmented_ita,empirical_class_segmented_ita,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class

In [3]:
df_test = pd.read_csv('../../datasets/fitzpatrick17k/fitzpatrick17k_train.csv')
print("Número de instâncias com -1 no label:"+str(len(df_test[df_test['fitzpatrick']== -1])))
print("Dropando estas instâncias...")
df_test = df_test.drop(df_test[df_test['fitzpatrick']== -1].index,axis=0)
df_test

Número de instâncias com -1 no label:503
Dropando estas instâncias...


,md5hash,fitzpatrick,label,nine_partition_label,three_partition_label,url,url_alphanum
0,aa4f94e33bbc118e2a95f7a8ad0344a1,2,folliculitis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicffolli...
1,b500453ca3910a06c8ff290dd7e3efa0,1,factitial dermatitis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicffacti...
2,c7cd549009d2ea6752de4fc4c2c02bc2,2,folliculitis,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=6170,httpwwwatlasdermatologicocombrimgimageId6170.jpg
4,ce925bdb440a53105f4fc46647c8fa7c,4,erythema multiforme,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpiceeryth...
5,15447b78321c1555992bccdb9a6aa54c,5,squamous cell carcinoma,malignant epidermal,malignant,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicssquam...
...,...,...,...,...,...,...,...
14866,a809a644cd3dca9d915a5f92b7c31405,3,sarcoidosis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicssarco...
14867,4c68f5b525f27790c42facbbda3abf5f,4,ehlers danlos syndrome,genodermatoses,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=1608,httpwwwatlasdermatologicocombrimgimageId1608.jpg
14868,d34824713baf81e9ec104e6c1d8b9791,2,psoriasis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
14870,f38ef800ff2e091ad6af7e233c0afa29,2,urticaria pigmentosa,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicuurtic...


In [4]:
image_dir = '../../datasets/fitzpatrick17k/images'

In [5]:
df_result = pd.DataFrame()
df_result

""


In [6]:
start = time.time()
result = df_test.apply(lambda row: func(os.path.join(image_dir,row['md5hash']+'.jpg'),row['fitzpatrick']),axis=1)
stop = time.time()
stop-start

/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:39: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(L), np.nanmean(L)
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:44: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(B), np.nanmean(B)
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:52: RuntimeWarning: Mean of empty slice
  ITA = math.atan2(np.nanmean(L) - 50, np.nanmean(B)) * (180 / np.pi)


4185.40224313736

In [7]:
df_aux = pd.DataFrame(list(result))
df_aux

,0,1,2,3,4,5,6,7,8
0,33.230388,32.043570,3,2,0,1,1,1,2
1,NaN,164.233651,6,6,0,0,0,0,1
2,48.519973,35.469942,2,1,1,0,1,1,2
3,-5.196491,9.876615,6,5,0,0,0,1,4
4,33.253809,55.188757,3,2,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...
14364,29.449512,29.519689,3,2,1,0,1,1,3
14365,38.138477,29.071445,3,2,0,0,1,0,4
14366,53.264299,52.827444,2,1,1,0,1,1,2
14367,38.879170,38.797381,3,2,0,1,1,1,2


In [8]:
df_result['segmented_ita'] = df_aux[0]
df_result['direct_ita'] = df_aux[1]
df_result['kinyananjui_class'] = df_aux[2]
df_result['empirical_class'] = df_aux[3]
df_result['kinyananjui_hit'] = df_aux[4]
df_result['empirical_hit'] = df_aux[5]
df_result['kinyananjui_relaxed_hit'] = df_aux[6]
df_result['empirical_relaxed_hit'] = df_aux[7]
df_result['real_class'] = df_aux[8]



df_result

,segmented_ita,direct_ita,kinyananjui_class,empirical_class,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class
0,33.230388,32.043570,3,2,0,1,1,1,2
1,NaN,164.233651,6,6,0,0,0,0,1
2,48.519973,35.469942,2,1,1,0,1,1,2
3,-5.196491,9.876615,6,5,0,0,0,1,4
4,33.253809,55.188757,3,2,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...
14364,29.449512,29.519689,3,2,1,0,1,1,3
14365,38.138477,29.071445,3,2,0,0,1,0,4
14366,53.264299,52.827444,2,1,1,0,1,1,2
14367,38.879170,38.797381,3,2,0,1,1,1,2


In [9]:
df_result.to_csv('baseline_results_train_split.csv',index=False)